In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('./zomato.csv', encoding='latin1')
data

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,NamlÛ± Gurme,208,ÛÁstanbul,"Kemankeô Karamustafa Paôa Mahallesi, RÛ±htÛ±...",Karakí_y,"Karakí_y, ÛÁstanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz AÛôacÛ±,208,ÛÁstanbul,"Koôuyolu Mahallesi, Muhittin íìstí_ndaÛô Cadd...",Koôuyolu,"Koôuyolu, ÛÁstanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,Aôôk Kahve,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [4]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [6]:
data[data['City'].str.contains('Delhi')][['Restaurant ID', 'Restaurant Name', 'City','Cuisines',
                                              'Average Cost for two', 'Currency', 'Has Table booking',
                                              'Has Online delivery', 'Is delivering now',  'Aggregate rating', 'Rating color', 'Rating text',
                                              'Votes']]


,Restaurant ID,Restaurant Name,City,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Aggregate rating,Rating color,Rating text,Votes
2560,18287358,Food Cloud,New Delhi,Cuisine Varies,500,Indian Rupees(Rs.),No,No,No,0.0,White,Not rated,2
2561,18216944,Burger.in,New Delhi,Fast Food,350,Indian Rupees(Rs.),No,Yes,No,3.2,Orange,Average,46
2562,313333,Days of the Raj,New Delhi,"North Indian, Seafood, Continental",1500,Indian Rupees(Rs.),Yes,Yes,No,3.4,Orange,Average,45
2563,18384127,Dilli Ka Dhaba,New Delhi,"South Indian, North Indian",500,Indian Rupees(Rs.),No,No,No,2.6,Orange,Average,11
2564,582,Govardhan,New Delhi,"South Indian, North Indian, Chinese",500,Indian Rupees(Rs.),No,Yes,No,3.4,Orange,Average,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,18445274,Motu N Patlu,New Delhi,"Lebanese, North Indian, Fast Food",250,Indian Rupees(Rs.),No,No,No,0.0,White,Not rated,0
8029,18168147,S.K. Fast Food,New Delhi,Chinese,350,Indian Rupees(Rs.),No,No,No,0.0,White,Not rated,0
8030,18429375,Sufiyan Restaurant,New Delhi,Mughlai,250,Indian Rupees(Rs.),No,No,No,0.0,White,Not rated,0
8031,310479,The Relax Point,New Delhi,North Indian,250,Indian Rupees(Rs.),No,No,No,0.0,White,Not rated,1


In [4]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [5]:
dict1 = {
    162: 'Philippines',
    30: 'Brazil',
    14: 'Australia',
    216: 'USA',
    37: 'Canada',
    184: 'Singapore',
    214: 'UAE',
    1: 'India',
    94: 'Indonesia',
    148: 'New Zealand',
    215: 'UK',
    166: 'Qatar',
    189: 'South Africa',
    191: 'Sri Lanka',
    208: 'Turkey'
}

In [19]:
data['Country'] = data['Country Code'].map(dict1)

In [7]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes', 'Country'],
      dtype='object')

In [20]:
selected_features = ['Restaurant ID', 'Restaurant Name','Country','City',
            'Average Cost for two', 'Has Table booking',
       'Has Online delivery','Aggregate rating', 'Votes']

In [21]:
data = data[selected_features]

In [22]:
data.to_csv('restaurants.csv', index = False)

In [23]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country', 'City',
       'Average Cost for two', 'Has Table booking', 'Has Online delivery',
       'Aggregate rating', 'Votes'],
      dtype='object')

In [11]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country', 'Locality',
       'Average Cost for two', 'Has Table booking', 'Has Online delivery',
       'Price range', 'Aggregate rating', 'Votes'],
      dtype='object')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [15]:
data.dropna(inplace=True)

In [11]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [12]:
selected_features = ['Restaurant ID', 'Restaurant Name', 'Country Code', 'City',
       'Locality', 'Cuisines', 'Average Cost for two', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu','Aggregate rating','Votes']

In [13]:
data[data['Country Code'] == 162]

KeyError: 'Country Code'

In [ ]:
data['Country Code'].unique()


In [16]:
data['Country Code'].unique()

array([162,  30, 216,  14,  37, 184, 214,   1,  94, 148, 215, 166, 189,
       191, 208], dtype=int64)

In [ ]:
country_name = {
    162: 'Christmas Island', 30: 'Brazil', 216: 'USA',
    14: 'Australia', 37: , 184: , 214,   1,  94, 148, 215, 166, 189,
       191, 208
}

In [21]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [20]:
data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [13]:
df = data[selected_features]

In [ ]:
df = data[]

In [25]:
data = pd.read_csv('./zomato.csv', encoding='latin1')
data.dropna(inplace = True)

In [26]:
# Remove irrelevant columns
data = data[['Restaurant ID', 'Restaurant Name', 'City', 'Locality', 'Cuisines', 'Price range', 'Aggregate rating']]

# Handle missing values in 'Cuisines' column
data['Cuisines'].fillna('Unknown', inplace=True)

# Step 2: Feature selection1

# Select relevant features
selected_features = ['City', 'Locality', 'Cuisines', 'Price range', 'Aggregate rating']

# Step 3: Create a feature vector
data['Features'] = data[selected_features].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Step 4: Vectorize the feature vector
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(data['Features'])
print(feature_vectors)
# Step 5: Calculate similarity
similarity_matrix = cosine_similarity(feature_vectors)

# Step 6: Recommend restaurants
def recommend_restaurants(user_preferences, top_n=5):
    # Create a user profile based on preferences
    user_profile = ' '.join(user_preferences)
    user_vector = vectorizer.transform([user_profile])

    # Calculate similarity between user profile and all restaurants
    similarity_scores = cosine_similarity(user_vector, feature_vectors).flatten()

    # Sort restaurants based on similarity scores
    restaurant_indices = np.argsort(similarity_scores)[::-1][:top_n]

    # Get top N recommended restaurants
    recommended_restaurants = data.iloc[restaurant_indices]

    return recommended_restaurants

# Example usage
user_preferences = ['New York', 'Manhattan', 'Italian', '2', '4.5']
recommendations = recommend_restaurants(user_preferences)

print(recommendations[['Restaurant Name', 'City', 'Cuisines', 'Aggregate rating']])


  (0, 419)	0.13046032660555462
  (0, 706)	0.18612781756804514
  (0, 521)	0.23970412680678782
  (0, 1156)	0.33569580015336303
  (0, 909)	0.13405077874972635
  (0, 307)	0.33569580015336303
  (0, 340)	0.48428369510699215
  (0, 905)	0.642646731252887
  (1, 1514)	0.23758159018135525
  (1, 853)	0.34750677995041945
  (1, 1442)	0.34750677995041945
  (1, 866)	0.33262867173394406
  (1, 706)	0.19267646039277536
  (1, 340)	0.3342150192569924
  (1, 905)	0.6652573434678881
  (2, 663)	0.0646800964780958
  (2, 505)	0.2780728725600226
  (2, 147)	0.16843993300762025
  (2, 1294)	0.1789961087227963
  (2, 1075)	0.30630593708142473
  (2, 819)	0.28947605704370477
  (2, 1311)	0.28947605704370477
  (2, 469)	0.33911645627223286
  (2, 915)	0.6126118741628495
  (2, 340)	0.32614561643821105
  :	:
  (9536, 272)	0.3039797681965496
  (9537, 753)	0.6419616169967719
  (9537, 1577)	0.5595140377070911
  (9537, 1458)	0.5242416655623606
  (9538, 1576)	0.4514906334012487
  (9538, 791)	0.4514906334012487
  (9538, 1577)	0.355

In [8]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'City', 'Locality', 'Cuisines',
       'Price range', 'Aggregate rating', 'Features'],
      dtype='object')